This notebook calculates the values for the tables in the SI, as well as for figure 3 in the main text. To avoid confusion, please note that aprcpm is piControl monthly precipitation, and cprcpm is the hosing experiment monthly precipitation (see txt file on naming conventions).

# Utilities

In [7]:
import numpy as np
import xarray as xr
import glob
from functions import *
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
from scipy.stats import pearsonr

In [2]:
model_names = ['HadGEM3-GC3-1MM','CanESM5','CESM2','IPSL-CM6A-LR']
model_names2 = ['HadGEM3-GC31-MM','CanESM5','CESM2','IPSL-CM6A-LR']

In [3]:
prcp_keys = dict(zip(model_names,['precipitation_flux','pr','PRECC','pr']))

In [4]:
aidcs = [[0,80*12],[0,80*12],[0+11,80*12+11],[0,80*12]]
cidcs = [[20*12,100*12],[20*12,100*12],[100*12+11,180*12+11],[60*12,140*12]]
aslices1 = dict(zip(model_names,aidcs))
cslices1 = dict(zip(model_names,cidcs))
aslices = aslices1
cslices = cslices1
aslicesm = aslices1
cslicesm = cslices1

In [12]:
aidcs = [[0,80],[0,80],[1,81],[0,80]]
cidcs = [[20,100],[20,100],[101,181],[60,140]]
aslicesy = dict(zip(model_names,aidcs))
cslicesy = dict(zip(model_names,cidcs))

In [5]:
idc = [[60,140],[60,140],[60,140],[60,140]]
co2_slicesy = dict(zip(model_names,idc))
idc = [[60*12,140*12],[60*12,140*12],[60*12,140*12],[60*12,140*12]]
co2_slicesm = dict(zip(model_names,idc))

In [6]:
boxes_latlon = [[285-360,317.5-360,-15,-5,],
    [295-360,315-360,-15,-5],
    [-75,-60,0,10],
    [-75,-50,-10,0],
    [-50,-40,-15,-5],
    [-20,20,9,20],
    [-20,20,0,10],
    [-20,20,5,15],
   [-75,-55,-5,5],
   [-20,25,5,20],
   [70,85,5,25],
    [110,140,10,40]]
boxes_dict = {
    'am1': [285,317.5,-15,-5],
    'am2' :[295,315,-15,-5],
     'pam1':   [360-75,360-60,0,10],
    'pam2': [360-75,360-50,-10,0],
    'pam3': [360-50,360-40,-15,-5],
    'wam1': [360-20,360+20,9,20],
    'wam2': [360-20,360+20,0,10],
    'wam3': [360-20,360+20,5,15],
    'am3':[360-75,360-55,-5,5],
    'wam4':[360-20,25,5,20],
    'ism1':[70,85,5,25],
    'easm2':[110,140,10,40]
}

# all model agreement in different scenarios

## geographic

In [9]:
## agreement in NAHosMIP
diff_prcp_Cgrid = {}

for model in model_names:
    adata = xr.open_dataset('/p/tmp/mayayami/NAHosMIP/{}/vars/aprcpm_Cgrid.nc'.format(model))[prcp_keys[model]][aslicesm[model][0]:aslicesm[model][1]]
    cdata = xr.open_dataset('/p/tmp/mayayami/NAHosMIP/{}/vars/cprcpm_Cgrid.nc'.format(model))[prcp_keys[model]][cslicesm[model][0]:cslicesm[model][1]]

    diff_prcp_Cgrid[model] = (cdata.mean(dim='time'))-(adata.mean(dim='time'))

hsign = np.sign(diff_prcp_Cgrid['HadGEM3-GC3-1MM'])
match = hsign.where(
                      (np.sign(diff_prcp_Cgrid['CanESM5'])==hsign)&
                    (np.sign(diff_prcp_Cgrid['CESM2'])==hsign)&
                    (np.sign(diff_prcp_Cgrid['IPSL-CM6A-LR'])==hsign)
)

/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


In [16]:
# agreement in 4xco2 experiments
co2_diff_prcp_Cgrid = {}

for model in model_names:
    adata = xr.open_dataset('/p/tmp/mayayami/NAHosMIP/{}/vars/aprcpm_Cgrid.nc'.format(model))[prcp_keys[model]][aslicesm[model][0]:aslicesm[model][1]]
    co2_data = (xr.open_dataset('/p/tmp/mayayami/NAHosMIP/{}/4CO2_prcp_Cgrid_ymean.nc'.format(model)).pr*86400)[co2_slicesy[model][0]:co2_slicesy[model][1]]

    co2_diff_prcp_Cgrid[model] = (co2_data.mean(dim='time'))-(adata.mean(dim='time'))

co2_hsign = np.sign(co2_diff_prcp_Cgrid['HadGEM3-GC3-1MM'])
co2_match = co2_hsign.where(
                      (np.sign(co2_diff_prcp_Cgrid['CanESM5'])==co2_hsign)&
                    (np.sign(co2_diff_prcp_Cgrid['CESM2'])==co2_hsign)&
                    (np.sign(co2_diff_prcp_Cgrid['IPSL-CM6A-LR'])==co2_hsign)
)

/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)
/home/mayayami/.conda/envs/main/lib/python3.9/si

In [19]:
# agreement in ssp585
ssp585_diff_prcp_Cgrid = {}

for i, model in enumerate(model_names):
    adata = xr.open_dataset('/p/tmp/mayayami/NAHosMIP/{}/vars/aprcpm_Cgrid.nc'.format(model_names[i]))[prcp_keys[model_names[i]]][aslicesm[model_names[i]][0]:aslicesm[model_names[i]][1]]
    ifile = glob.glob('/p/tmp/mayayami/NAHosMIP/scenario_pr/pr_Amon_{}_ssp585_*_ymean_Cgrid.nc'.format(model_names2[i]))[0]
    ssp585_data = (xr.open_dataset(ifile).pr*86400)[66:86]

    ssp585_diff_prcp_Cgrid[model] = (ssp585_data.mean(dim='time'))-(adata.mean(dim='time'))

ssp585_hsign = np.sign(ssp585_diff_prcp_Cgrid['HadGEM3-GC3-1MM'])
ssp585_match = ssp585_hsign.where(
                      (np.sign(ssp585_diff_prcp_Cgrid['CanESM5'])==ssp585_hsign)&
                    (np.sign(ssp585_diff_prcp_Cgrid['CESM2'])==ssp585_hsign)&
                    (np.sign(ssp585_diff_prcp_Cgrid['IPSL-CM6A-LR'])==ssp585_hsign)
)

/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)
/home/mayayami/.conda/envs/main/lib/python3.9/si

In [20]:
# agreement in ssp126
ssp126_diff_prcp_Cgrid = {}

for i, model in enumerate(model_names):
    adata = xr.open_dataset('/p/tmp/mayayami/NAHosMIP/{}/vars/aprcpm_Cgrid.nc'.format(model_names[i]))[prcp_keys[model_names[i]]][aslicesm[model_names[i]][0]:aslicesm[model_names[i]][1]]
    ifile = glob.glob('/p/tmp/mayayami/NAHosMIP/scenario_pr/pr_Amon_{}_ssp126_*_ymean_Cgrid.nc'.format(model_names2[i]))[0]
    ssp126_data = (xr.open_dataset(ifile).pr*86400)[66:86]

    ssp126_diff_prcp_Cgrid[model] = (ssp126_data.mean(dim='time'))-(adata.mean(dim='time'))

ssp126_hsign = np.sign(ssp126_diff_prcp_Cgrid['HadGEM3-GC3-1MM'])
ssp126_match = ssp126_hsign.where(
                      (np.sign(ssp126_diff_prcp_Cgrid['CanESM5'])==ssp126_hsign)&
                    (np.sign(ssp126_diff_prcp_Cgrid['CESM2'])==ssp126_hsign)&
                    (np.sign(ssp126_diff_prcp_Cgrid['IPSL-CM6A-LR'])==ssp126_hsign)
)

/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)
/home/mayayami/.conda/envs/main/lib/python3.9/si

In [21]:
weights = xr.open_dataset('/p/tmp/mayayami/NAHosMIP/Cgrid_remap_areas.nc').cell_area
weights = weights/weights.mean()

In [22]:
# mask all areas to get only land
seamask = xr.open_dataset('/p/tmp/mayayami/NAHosMIP/{}/seamask_Cgrid.nc'.format('CanESM5')).topo
match_cont = match.where(seamask>=1)
co2_match_cont = co2_match.where(seamask>=1)
ssp585_match_cont = ssp585_match.where(seamask>=1)
ssp126_match_cont = ssp126_match.where(seamask>=1)
hsign_cont = hsign.where(seamask>=1)
co2_hsign_cont = co2_hsign.where(seamask>=1)
ssp585_hsign_cont = ssp585_hsign.where(seamask>=1)
ssp126_hsign_cont = ssp126_hsign.where(seamask>=1)

In [23]:
# gives 1 in all cells where there is data and nan otherwise
def fill(data):
    ds = xr.full_like(data,1)
    ds = ds.where(~data.isnull())
    return ds

In [24]:
# we want to count all the gridcells where the models match, so we fill each of those cells with 1
# we can then choose a region and add up the gridcells
match_cont = fill(match_cont)
co2_match_cont = fill(co2_match_cont)
ssp585_match_cont = fill(ssp585_match_cont)
ssp126_match_cont = fill(ssp126_match_cont)

In [25]:
def frac(d,d_tot,weights):
    n = (d*weights).sum() # weighted area where models agree
    n_tot = (d_tot*weights).sum() # weighted total area
    return n/n_tot

In [26]:
region_names = ['Global','No Poles','Extratropics','Tropics','S AM','N AM','WAM','ISM','EASM']

In [27]:
# functions for getting the frractions in a given region
def co2_reg_frac(min_lon,max_lon,min_lat,max_lat):
    wghts = get_box2(weights,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    mtch = get_box2(co2_match_cont,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    tot = get_box2(co2_hsign_cont,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    return frac(mtch,fill(tot),wghts).values
def ssp585_reg_frac(min_lon,max_lon,min_lat,max_lat):
    wghts = get_box2(weights,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    mtch = get_box2(ssp585_match_cont,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    tot = get_box2(ssp585_hsign_cont,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    return frac(mtch,fill(tot),wghts).values
def ssp126_reg_frac(min_lon,max_lon,min_lat,max_lat):
    wghts = get_box2(weights,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    mtch = get_box2(ssp126_match_cont,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    tot = get_box2(ssp126_hsign_cont,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    return frac(mtch,fill(tot),wghts).values
def reg_frac(min_lon,max_lon,min_lat,max_lat):
    wghts = get_box2(weights,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    mtch = get_box2(match_cont,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    tot = get_box2(hsign_cont,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    return frac(mtch,fill(tot),wghts).values

In [33]:
region_frac = []
co2_region_frac = []
ssp585_region_frac = []
ssp126_region_frac = []

# global
region_frac.append(frac(match_cont,fill(hsign_cont),weights).values)
co2_region_frac.append(frac(co2_match_cont,fill(co2_hsign_cont),weights).values)
ssp585_region_frac.append(frac(ssp585_match_cont,fill(ssp585_hsign_cont),weights).values)
ssp126_region_frac.append(frac(ssp126_match_cont,fill(ssp126_hsign_cont),weights).values)

# no poles
min_lon,max_lon,min_lat,max_lat = 0, 360, -60,60
region_frac.append(reg_frac(min_lon,max_lon,min_lat,max_lat))
co2_region_frac.append(co2_reg_frac(min_lon,max_lon,min_lat,max_lat))
ssp585_region_frac.append(ssp585_reg_frac(min_lon,max_lon,min_lat,max_lat))
ssp126_region_frac.append(ssp126_reg_frac(min_lon,max_lon,min_lat,max_lat))

# extratropics --> need to add up two regions
min_lon1,max_lon1,min_lat1,max_lat1 = 0, 360, 30,60
min_lon2,max_lon2,min_lat2,max_lat2 = 0, 360, -60,-30
region_frac.append((reg_frac(min_lon1,max_lon1,min_lat1,max_lat1)+reg_frac(min_lon2,max_lon2,min_lat2,max_lat2))/2)
co2_region_frac.append((co2_reg_frac(min_lon1,max_lon1,min_lat1,max_lat1)+co2_reg_frac(min_lon2,max_lon2,min_lat2,max_lat2))/2)
ssp585_region_frac.append((ssp585_reg_frac(min_lon1,max_lon1,min_lat1,max_lat1)+ssp585_reg_frac(min_lon2,max_lon2,min_lat2,max_lat2))/2)
ssp126_region_frac.append((ssp126_reg_frac(min_lon1,max_lon1,min_lat1,max_lat1)+ssp126_reg_frac(min_lon2,max_lon2,min_lat2,max_lat2))/2)

#tropics
min_lon,max_lon,min_lat,max_lat = 0, 360, -20,20
region_frac.append(reg_frac(min_lon,max_lon,min_lat,max_lat))
co2_region_frac.append(co2_reg_frac(min_lon,max_lon,min_lat,max_lat))
ssp585_region_frac.append(ssp585_reg_frac(min_lon,max_lon,min_lat,max_lat))
ssp126_region_frac.append(ssp126_reg_frac(min_lon,max_lon,min_lat,max_lat))

#boxes
for i, ibox in enumerate([0,8,9,10,11]):
    box = list(boxes_dict.keys())[ibox]
    latlon = boxes_dict[box]
    min_lon = latlon[0]
    max_lon = latlon[1]
    min_lat = latlon[2]
    max_lat = latlon[3]
    region_frac.append(reg_frac(min_lon,max_lon,min_lat,max_lat))
    co2_region_frac.append(co2_reg_frac(min_lon,max_lon,min_lat,max_lat))
    ssp585_region_frac.append(ssp585_reg_frac(min_lon,max_lon,min_lat,max_lat))
    ssp126_region_frac.append(ssp126_reg_frac(min_lon,max_lon,min_lat,max_lat))


In [38]:
df = pd.DataFrame(data=np.transpose(np.stack((region_frac,co2_region_frac,ssp585_region_frac,ssp126_region_frac))),index=region_names,columns = ['Hosing','4xCO2','SSP585','SSP126'])
display(df)

,Hosing,4xCO2,SSP585,SSP126
Global,0.66,0.65,0.66,0.60
No Poles,0.66,0.57,0.58,0.51
Extratropics,0.49,0.74,0.68,0.62
Tropics,0.64,0.40,0.53,0.40
S AM,0.79,0.15,0.27,0.03
N AM,0.18,0.68,0.61,0.36
WAM,0.99,0.15,0.48,0.24
ISM,0.95,0.75,0.80,0.74
EASM,0.71,0.73,0.96,0.70


In [39]:
df.to_csv('geographic_agreement.txt')

## monthly

In [23]:
mods = []
for im, model in enumerate(model_names):        
    adata = mavg(xr.open_dataset('/p/tmp/mayayami/NAHosMIP/{}/vars/aprcpm_Cgrid.nc'.format(model))[prcp_keys[model]][aslices[model][0]:aslices[model][1]])
    cdata = mavg(xr.open_dataset('/p/tmp/mayayami/NAHosMIP/{}/vars/cprcpm_Cgrid.nc'.format(model))[prcp_keys[model]][cslices[model][0]:cslices[model][1]])
    diff = cdata - adata
    sgn = np.sign(diff)
    mods.append(sgn)
sn = mods[0]
ag = mods[0].where((mods[1]==sn) & (mods[2]==sn) & (mods[3]==sn))
mon_frac = ag.count(dim='month').fillna(0)

/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


In [26]:
mon_frac.to_netcdf('/p/tmp/mayayami/NAHosMIP/all_monthly_match.nc')

In [49]:
mods = []
for im, model in enumerate(model_names):        
    adata = mavg(xr.open_dataset('/p/tmp/mayayami/NAHosMIP/{}/vars/aprcpm_Cgrid.nc'.format(model))[prcp_keys[model]][aslicesm[model][0]:aslicesm[model][1]])
    cdata = mavg(xr.open_dataset('/p/tmp/mayayami/NAHosMIP/{}/4CO2_prcp_Cgrid.nc'.format(model)).pr[co2_slicesm[model][0]:co2_slicesm[model][1]]*86400)
    diff = cdata - adata
    sgn = np.sign(diff)
    mods.append(sgn)
sn = mods[0]
ag_co2 = mods[0].where((mods[1]==sn) & (mods[2]==sn) & (mods[3]==sn))
frac_co2 = ag_co2.count(dim='month')
mon_frac_co2 = frac_co2.fillna(0)

/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)
/home/mayayami/.conda/envs/main/lib/python3.9/si

In [50]:
mods = []
for im, model in enumerate(model_names):        
    adata = mavg(xr.open_dataset('/p/tmp/mayayami/NAHosMIP/{}/vars/aprcpm_Cgrid.nc'.format(model))[prcp_keys[model]][aslicesm[model][0]:aslicesm[model][1]])
    ifiles = glob.glob('/p/tmp/mayayami/NAHosMIP/scenario_pr/pr_Amon_{}_ssp585_*_Cgrid.nc'.format(model_names2[im]))
    for file in ifiles:
        if 'ymean' not in file:
            ifile=file
    cdata = mavg((xr.open_dataset(ifile).pr*86400)[66*12:86*12])
    diff = cdata - adata
    sgn = np.sign(diff)
    mods.append(sgn)
sn = mods[0]
ag_ssp585 = mods[0].where((mods[1]==sn) & (mods[2]==sn) & (mods[3]==sn))
frac_ssp585 = ag_ssp585.count(dim='month')
mon_frac_ssp585 = frac_ssp585.fillna(0)

/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)
/home/mayayami/.conda/envs/main/lib/python3.9/si

In [51]:
mods = []
for im, model in enumerate(model_names):        
    adata = mavg(xr.open_dataset('/p/tmp/mayayami/NAHosMIP/{}/vars/aprcpm_Cgrid.nc'.format(model))[prcp_keys[model]][aslicesm[model][0]:aslicesm[model][1]])
    ifiles = glob.glob('/p/tmp/mayayami/NAHosMIP/scenario_pr/pr_Amon_{}_ssp126_*_Cgrid.nc'.format(model_names2[im]))
    for file in ifiles:
        if 'ymean' not in file:
            ifile=file
    print(ifile)
    cdata = mavg((xr.open_dataset(ifile).pr*86400)[66*12:86*12])
    diff = cdata - adata
    sgn = np.sign(diff)
    mods.append(sgn)
sn = mods[0]
ag_ssp126 = mods[0].where((mods[1]==sn) & (mods[2]==sn) & (mods[3]==sn))
frac_ssp126 = ag_ssp126.count(dim='month')
mon_frac_ssp126 = frac_ssp126.fillna(0)

/p/tmp/mayayami/NAHosMIP/scenario_pr/pr_Amon_HadGEM3-GC31-MM_ssp126_r1i1p1f3_gn_Cgrid.nc
/p/tmp/mayayami/NAHosMIP/scenario_pr/pr_Amon_CanESM5_ssp126_r1i1p1f1_gn_Cgrid.nc
/p/tmp/mayayami/NAHosMIP/scenario_pr/pr_Amon_CESM2_ssp126_r4i1p1f1_gn_Cgrid.nc


/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


/p/tmp/mayayami/NAHosMIP/scenario_pr/pr_Amon_IPSL-CM6A-LR_ssp126_r1i1p1f1_gr_Cgrid.nc


/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


In [52]:
def co2_reg_mon_frac(min_lon,max_lon,min_lat,max_lat):
    wghts = get_box2(weights,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    wghts = wghts/wghts.mean()
    frcs = get_box2(mon_frac_co2_cont,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    return (wghts*frcs).mean().values/12
def ssp585_reg_mon_frac(min_lon,max_lon,min_lat,max_lat):
    wghts = get_box2(weights,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    wghts = wghts/wghts.mean()
    frcs = get_box2(mon_frac_ssp585_cont,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    return (wghts*frcs).mean().values/12
def ssp126_reg_mon_frac(min_lon,max_lon,min_lat,max_lat):
    wghts = get_box2(weights,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    wghts = wghts/wghts.mean()
    frcs = get_box2(mon_frac_ssp126_cont,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    return (wghts*frcs).mean().values/12
def reg_mon_frac(min_lon,max_lon,min_lat,max_lat):
    wghts = get_box2(weights,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    wghts = wghts/wghts.mean()
    frcs = get_box2(mon_frac_cont,min_lon,max_lon,min_lat,max_lat,lon=True,drop=True)
    return (wghts*frcs).mean().values/12

In [53]:
seamask = xr.open_dataset('/p/tmp/mayayami/NAHosMIP/{}/seamask_Cgrid.nc'.format('CanESM5')).topo
mon_frac_cont = mon_frac*seamask
mon_frac_co2_cont = mon_frac_co2*seamask
mon_frac_ssp585_cont = mon_frac_ssp585*seamask
mon_frac_ssp126_cont = mon_frac_ssp126*seamask

In [54]:
mon_region_frac = []
mon_co2_region_frac = []
mon_ssp585_region_frac = []
mon_ssp126_region_frac = []

# global
mon_region_frac.append((weights*mon_frac).mean().values/12)
mon_co2_region_frac.append((weights*mon_frac_co2).mean().values/12)
mon_ssp585_region_frac.append((weights*mon_frac_ssp585).mean().values/12)
mon_ssp126_region_frac.append((weights*mon_frac_ssp126).mean().values/12)

# no poles
min_lon,max_lon,min_lat,max_lat = 0, 360, -60,60
mon_region_frac.append(reg_mon_frac(min_lon,max_lon,min_lat,max_lat))
mon_co2_region_frac.append(co2_reg_mon_frac(min_lon,max_lon,min_lat,max_lat))
mon_ssp585_region_frac.append(ssp585_reg_mon_frac(min_lon,max_lon,min_lat,max_lat))
mon_ssp126_region_frac.append(ssp126_reg_mon_frac(min_lon,max_lon,min_lat,max_lat))

# extratropics
min_lon1,max_lon1,min_lat1,max_lat1 = 0, 360, 30,60
min_lon2,max_lon2,min_lat2,max_lat2 = 0, 360, -60,-30
mon_region_frac.append((reg_mon_frac(min_lon1,max_lon1,min_lat1,max_lat1)+reg_mon_frac(min_lon2,max_lon2,min_lat2,max_lat2))/2)
mon_co2_region_frac.append((co2_reg_mon_frac(min_lon1,max_lon1,min_lat1,max_lat1)+co2_reg_mon_frac(min_lon2,max_lon2,min_lat2,max_lat2))/2)
mon_ssp585_region_frac.append((ssp585_reg_mon_frac(min_lon1,max_lon1,min_lat1,max_lat1)+ssp585_reg_mon_frac(min_lon2,max_lon2,min_lat2,max_lat2))/2)
mon_ssp126_region_frac.append((ssp126_reg_mon_frac(min_lon1,max_lon1,min_lat1,max_lat1)+ssp126_reg_mon_frac(min_lon2,max_lon2,min_lat2,max_lat2))/2)

#tropics
min_lon,max_lon,min_lat,max_lat = 0, 360, -30,30
mon_region_frac.append(reg_mon_frac(min_lon,max_lon,min_lat,max_lat))
mon_co2_region_frac.append(co2_reg_mon_frac(min_lon,max_lon,min_lat,max_lat))
mon_ssp585_region_frac.append(ssp585_reg_mon_frac(min_lon,max_lon,min_lat,max_lat))
mon_ssp126_region_frac.append(ssp126_reg_mon_frac(min_lon,max_lon,min_lat,max_lat))

#boxes
for i, ibox in enumerate([0,8,9,10,11]):
    box = list(boxes_dict.keys())[ibox]
    latlon = boxes_dict[box]
    min_lon = latlon[0]
    max_lon = latlon[1]
    min_lat = latlon[2]
    max_lat = latlon[3]
    mon_region_frac.append(reg_mon_frac(min_lon,max_lon,min_lat,max_lat))
    mon_co2_region_frac.append(co2_reg_mon_frac(min_lon,max_lon,min_lat,max_lat))
    mon_ssp585_region_frac.append(ssp585_reg_mon_frac(min_lon,max_lon,min_lat,max_lat))
    mon_ssp126_region_frac.append(ssp126_reg_mon_frac(min_lon,max_lon,min_lat,max_lat))


In [55]:
df = pd.DataFrame(data=np.transpose(np.stack((mon_region_frac,mon_co2_region_frac,mon_ssp585_region_frac,mon_ssp126_region_frac))),index=region_names,columns = ['Hosing','4xCO2','SSP585','SSP126'])
display(df)

,Hosing,4xCO2,SSP585,SSP126
Global,0.35,0.52,0.49,0.33
No Poles,0.42,0.41,0.39,0.26
Extratropics,0.33,0.56,0.49,0.33
Tropics,0.40,0.31,0.30,0.21
S AM,0.60,0.05,0.09,0.08
N AM,0.73,0.32,0.30,0.15
WAM,0.65,0.17,0.21,0.13
ISM,0.44,0.30,0.34,0.12
EASM,0.26,0.50,0.36,0.21


In [56]:
df.to_csv('monthly_agreement.txt')

# correlation with HadGEM

In [13]:
aprcpy = {}
cprcpy = {}
diff_prcp = {}

for model in model_names:
    adata = xr.open_dataset('/p/tmp/mayayami/NAHosMIP/{}/vars/aprcpy.nc'.format(model))[prcp_keys[model]][aslicesy[model][0]:aslicesy[model][1]]
    cdata = xr.open_dataset('/p/tmp/mayayami/NAHosMIP/{}/vars/cprcpy.nc'.format(model))[prcp_keys[model]][cslicesy[model][0]:cslicesy[model][1]]
    
    aprcpy[model]=adata
    cprcpy[model]=cdata
    diff_prcp[model]=(cdata.mean(dim='time'))-(adata.mean(dim='time'))

/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/mayayami/.conda/envs/main/lib/python3.9/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


In [8]:
# sing HadGEM has the highest resolution grid, to correlate each model with HadGEM we have to regrid HadGEM to 
# the respective model grids
grds = ['C','CE','I']
aH_dgrid = {}
cH_dgrid = {}
diff_H_dgrid = {}

for i,gr in enumerate(grds):
    model = model_names[i+1]
    adata = xr.open_dataset('/p/tmp/mayayami/NAHosMIP/HadGEM3-GC3-1MM/vars/aprcpm_{}grid.nc'.format(gr))[prcp_keys['HadGEM3-GC3-1MM']][aslicesm['HadGEM3-GC3-1MM'][0]:aslicesm['HadGEM3-GC3-1MM'][1]]
    cdata = xr.open_dataset('/p/tmp/mayayami/NAHosMIP/HadGEM3-GC3-1MM/vars/cprcpm_{}grid.nc'.format(gr))[prcp_keys['HadGEM3-GC3-1MM']][cslicesm['HadGEM3-GC3-1MM'][0]:cslicesm['HadGEM3-GC3-1MM'][1]]

    aH_dgrid[model]=adata
    cH_dgrid[model]=cdata
    diff_H_dgrid[model] = (cdata.mean(dim='time'))-(adata.mean(dim='time'))


In [9]:
region_dict = {
        'am':[360-85,360-30,-20,11],
        'wam': [360-20,35,-10,25],
        'ism': [70, 85, 5, 25],
        'easm':[110,140,10,40]}

In [20]:
corrs = np.zeros((4,3,3)) # the axes are 0: regions, 1: models, 2: control/weak/diff
for ir, reg in enumerate(list(region_dict.keys())):
    region = region_dict[reg]
    for im, model in enumerate(model_names[1:]):
        ds1 = get_box3(aprcpy[model].mean(dim='time'),region,drop=True)
        ds2 = get_box3((aH_dgrid[model]).mean(dim='time'),region,drop=True)
        r = pearsonr(ds2.values.flatten(),ds1.values.flatten())
        corrs[ir,im,0]=r[0]
    for im, model in enumerate(model_names[1:]):
        ds1 = get_box3(cprcpy[model].mean(dim='time'),region,drop=True)
        ds2 = get_box3((cH_dgrid[model]).mean(dim='time'),region,drop=True)
        r = pearsonr(ds2.values.flatten(),ds1.values.flatten())
        corrs[ir,im,1]=r[0]
    for im, model in enumerate(model_names[1:]):
        ds1 = get_box3(diff_prcp[model],region,drop=True)
        ds2 = get_box3(diff_H_dgrid[model],region,drop=True)
        r = pearsonr(ds2.values.flatten(),ds1.values.flatten())
        corrs[ir,im,2]=r[0]


In [21]:
print('\n------Control-------')
df = pd.DataFrame(data=corrs[:,:,0],index=list(region_dict.keys()),columns=model_names[1:])
display(df)

print('\n------Weak AMOC-------')
df = pd.DataFrame(data=corrs[:,:,1],index=list(region_dict.keys()),columns=model_names[1:])
display(df)

print('\n------Difference-------')
df = pd.DataFrame(data=corrs[:,:,2],index=list(region_dict.keys()),columns=model_names[1:])
display(df)


------Control-------


,CanESM5,CESM2,IPSL-CM6A-LR
am,0.59,0.83,0.68
wam,0.90,0.93,0.88
ism,0.81,0.85,0.62
easm,0.82,0.81,0.73



------Weak AMOC-------


,CanESM5,CESM2,IPSL-CM6A-LR
am,0.56,0.82,0.67
wam,0.92,0.94,0.90
ism,0.89,0.91,0.70
easm,0.84,0.81,0.74



------Difference-------


,CanESM5,CESM2,IPSL-CM6A-LR
am,0.69,0.84,0.69
wam,0.81,0.90,0.77
ism,0.39,0.48,-0.09
easm,0.44,0.17,-0.00


In [22]:
df = pd.DataFrame(np.concatenate((corrs[:,0,:],corrs[:,1,:],corrs[:,2,:]),axis=1),index=region_dict.keys())
display(df)

,0,1,2,3,4,5,6,7,8
am,0.59,0.56,0.69,0.83,0.82,0.84,0.68,0.67,0.69
wam,0.90,0.92,0.81,0.93,0.94,0.90,0.88,0.90,0.77
ism,0.81,0.89,0.39,0.85,0.91,0.48,0.62,0.70,-0.09
easm,0.82,0.84,0.44,0.81,0.81,0.17,0.73,0.74,-0.00
